In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import unidecode
import os

In [2]:
soup = BeautifulSoup(open("../datasets/source_code_mutual_art.html"))
soup2 = BeautifulSoup(open("../datasets/artistas_seguidos_1.txt").read())
soup3 = BeautifulSoup(open("../datasets/artistas_seguidos_2.txt").read())
soup4 = BeautifulSoup(open("../datasets/artistas_seguidos_3.txt").read())
soup5 = BeautifulSoup(open("../datasets/artistas_seguidos_5.txt").read())
soup6 = BeautifulSoup(open("../datasets/artistas_seguidos_6.txt").read())
soup7 = BeautifulSoup(open("../datasets/artistas_seguidos_7.txt").read())
soup8 = BeautifulSoup(open("../datasets/paises_latam_1.txt").read())
soup9 = BeautifulSoup(open("../datasets/paises_latam_2.txt").read())
soup10 = BeautifulSoup(open("../datasets/paises_latam_3.txt").read())

soups = [soup,soup2,soup3,soup4,soup5,soup6,soup7,soup8,soup9,soup10]

In [3]:
precios_fecha=[['precio_estimado','precio_venta','fecha','autor','casa_subasta','tipo','tecnica','dimensiones','fecha_creacion','titulo','precio_ajustado']]
for soup in soups:
    obras = soup.findAll("div",{"class":"item table-row row v2__page"})
    for obra in obras:
        features=obra.findAll(lambda tag: tag.name=='div' and tag.get("class")==['v2__artwork-detail__price-col'])
        datos_obra=obra.findAll(lambda tag: tag.get("class")==['v2__link--primary'])
        tecnica = obra.findAll("span", {"itemprop": "artMedium"})
        dimensiones = obra.findAll("span", {"itemprop": "artDimensions"})
        fecha_creacion = obra.findAll("span", {"itemprop": "dateCreated"})
        titulo = obra.find("a", {"href": re.compile(r"/Artwork/.*")})
        if (len(features)==3 and len(datos_obra)==3 and len(tecnica)==1 and len(dimensiones)>=1):
            if (len(fecha_creacion) > 0):
                fecha_pintura = fecha_creacion[0].text
            else: 
                fecha_pintura = None
            precio_ajustado=re.findall(r"([0-9]+,?([0-9]+) [A-Z]+)",str(features[1]))
            if (len(precio_ajustado)>1):
                if (type(precio_ajustado[len(precio_ajustado)-2])==tuple):
                    precio_ajustado = precio_ajustado[len(precio_ajustado)-2][0]
                else:
                    precio_ajustado = precio_ajustado[len(precio_ajustado)-2]
            else: 
                precio_ajustado = None
            precio_estimado=features[0].text
            precio_venta=features[1].text
            fecha=features[2].text
            autor=datos_obra[0].text
            casa_subasta=datos_obra[1].text
            tipo=datos_obra[2].text
            tecnica=tecnica[0].text
            dimensiones=dimensiones[0].text
            titulo = titulo.find("img")['title']
            precios_fecha.append([precio_estimado,precio_venta,fecha,autor,casa_subasta,tipo,tecnica,dimensiones,fecha_pintura,titulo,precio_ajustado])
        else:
            pass

In [4]:
len(precios_fecha)

22298

In [5]:
data=pd.DataFrame(precios_fecha[1:],columns=precios_fecha[0])
for column in data.columns:
    data[str(column)]=data[str(column)].replace(r'\s+', ' ', regex=True)
    data[str(column)]=data[unidecode.unidecode(str(column))]
data.drop_duplicates(inplace=True)

In [6]:
data.shape

(20128, 11)

In [7]:
data[data.precio_ajustado.isna() != True].shape

(13646, 11)

In [8]:
data[(data.precio_venta != ' Upcoming ') & (data.precio_venta != ' Live Now ') & (data.precio_venta != ' Results Coming Soon ') & (data.precio_venta != ' Not Sold ')]

,precio_estimado,precio_venta,fecha,autor,casa_subasta,tipo,tecnica,dimensiones,fecha_creacion,titulo,precio_ajustado
862,700 EUR,700 EUR,"Sep 21, 2022",Jerson Jimenez,"Dorotheum, Vienna",Charity Auction of the Salvatorians,Oil on canvas,50 x 40 cm,2021,"""Experiencia"" by Jerson Jimenez, 2021",705 USD
975,"4,000 - 4,400 EUR","5,120 EUR","Sep 20, 2022",Julio Le Parc,"Dorotheum, Vienna",Modern and Contemporary Art,screenprint on cardboard,"41 x 41 x 5 cm, 8, 1970,",1970,"Relief 8 by Julio Le Parc, 1970","5,154 USD"
976,"1,000 - 1,500 GBP","1,275 GBP","Sep 20, 2022",Raúl Anguiano,Bonhams Knightsbridge,Travel & Exploration,oil on canvas,65.1 x 50.2cm (25 x 19 3/4in),1952,"Portrait of a young woman by Raúl Anguiano, 1952","1,485 USD"
977,"2,000 - 3,000 AUD","1,200 AUD*","Sep 20, 2022",Jose Haro,Ozbid Auctions,HUGE MODERN ART AUCTION - LIVE ONLINE,Mixed media,94cm x 153cm,None,Macaw by Jose Haro,824 USD
978,350 - 400 CHF,225 CHF*,"Sep 20, 2022",Jorge Camacho,TGP Auction,Pop & Contemporary Art,lithograph on Guarro paper,56 x 76 cm 56 x 76 cm 10 H. 10 H.,1974,"Cenizas by Jorge Camacho, 1974",231 USD
...,...,...,...,...,...,...,...,...,...,...,...
22288,"800 - 1,200 GBP",625 GBP,"Mar 14, 2009",Carolina Raquel Antich,Phillips London,Saturday@Phillips,Acrylic on canvas.,90 x 70 cm. (35 ½ x 27 ½ in.),2005,"Comparsa (Lei) by Carolina Raquel Antich, 2005","1,204 USD"
22289,"1,000 - 1,500 GBP",313 GBP,"Mar 14, 2009",Carolina Raquel Antich,Phillips London,Saturday@Phillips,Acrylic on canvas.,106 x 85 cm. (41 3/4 x 33 1/2 in.),2005,"Dipingere by Carolina Raquel Antich, 2005",603 USD
22291,100 - 150 GBP,100 GBP,"Mar 25, 2009",Axel Amuchastegui,Woolley & Wallis,"Oil paintings, Watercolours, Prints & Miniatur...",Watercolour heightened with bodycolour,25.5 x 21.5cm; 10 x 8½in,None,Racoon by Axel Amuchastegui,202 USD
22292,"6,500 - 7,000 EUR","7,500 EUR","Mar 24, 2009 - Mar 26, 2009",Adriano Spilimbergo,"Il Ponte Auction House, Via Pontaccio","Furnishing, Old Master Painting of the 19th Ce...",oil on canvas,cm 50x60,1963,"Venezia - San Giorgio by Adriano Spilimbergo, ...","13,994 USD"


In [9]:
df = data.copy()

In [10]:
#Normalización precio
df['precio_venta'] = df['precio_venta'].str.split(' ')
df['precio_venta'] = df['precio_venta'].apply(lambda x: x[1])
df['precio_venta'] = df['precio_venta'].str.replace(re.compile(r'^[A-Za-z]+'),'0')
df['precio_venta'] = df['precio_venta'].str.replace(',','')
df['precio_venta'] = df['precio_venta'].astype(float)
df['precio_estimado_'] = df['precio_estimado'].str.split(' ')
df['precio_estimado_1'] = df['precio_estimado_'].apply(lambda x: x[1])
df['precio_estimado_2'] = df['precio_estimado_'].apply(lambda x: x[3])
df['precio_estimado_1'] = df['precio_estimado_1'].str.replace(',','')
df['precio_estimado_2'] = df['precio_estimado_2'].str.replace(',','')
df['precio_estimado_1'] = df['precio_estimado_1'].str.replace(re.compile(r'^[A-Za-z]+'),'0')
df['precio_estimado_2'] = df['precio_estimado_2'].str.replace(re.compile(r'^[A-Za-z]+'),'0')
df['precio_estimado_2'] = df['precio_estimado_2'].str.replace(re.compile(r'^\s*$'),'0')
df['precio_estimado_1'] = df['precio_estimado_1'].astype(float)
df['precio_estimado_2'] = df['precio_estimado_2'].astype(float)
df['precio_ajustado'] = df['precio_ajustado'].str.replace(',','')
df['precio_ajustado'] = df['precio_ajustado'].str.replace(' USD','')
df['precio_ajustado'] = df['precio_ajustado'].str.replace(re.compile(r'^[A-Za-z]+'),'0')
df['precio_ajustado'] = df['precio_ajustado'].astype(float)
df['precio_inicio_ajustado'] = (df['precio_ajustado']/df['precio_venta'])*df['precio_estimado_1']
df['precio_fin_ajustado'] = (df['precio_ajustado']/df['precio_venta'])*df['precio_estimado_2']

In [11]:
df.drop(columns=['precio_estimado_1','precio_estimado_2','precio_estimado_'],inplace=True)

In [12]:
df.tail(20)

,precio_estimado,precio_venta,fecha,autor,casa_subasta,tipo,tecnica,dimensiones,fecha_creacion,titulo,precio_ajustado,precio_inicio_ajustado,precio_fin_ajustado
22277,"3,000 - 5,000 EUR",2750.0,"Mar 10, 2009",José (Tony) Cruz,Christie's Amsterdam,"NINETEEN TO NOW - ART FROM THE 19TH, 20TH & 21...",oil on board,55 x 46 cm,None,Portrait of a Spanish beauty by José (Tony) Cruz,4841.0,5281.090909,8801.818182
22278,"10,000 - 15,000 USD",0.0,"Mar 10, 2009",Enoc Perez,Sotheby's New York,Contemporary,oil on canvas,65 1/2 by 79 1/2 in. (alt: 166.4 by 202 cm.),None,Made in Mexico by Enoc Perez,NaN,NaN,NaN
22279,"20,000 - 30,000 USD",23750.0,"Mar 11, 2009",Vik Muniz,Christie's New York,FIRST OPEN POST-WAR AND CONTEMPORARY ART,print,41¼ x 34 in. (104.7 x 86.3 cm.),1999,"Mass II (Pictures of Chocolate) by Vik Muniz, ...",32787.0,27610.105263,41415.157895
22280,"40,000 - 60,000 USD",30000.0,"Mar 11, 2009",Vik Muniz,Christie's New York,FIRST OPEN POST-WAR AND CONTEMPORARY ART,print,60 x 45 in. (152.4 x 114.3 cm.),2001,After Van Gogh (Pictures of Color) by Vik Muni...,41415.0,55220.000000,82830.000000
22281,"15,000 - 20,000 USD",16250.0,"Mar 11, 2009",Vik Muniz,Christie's New York,FIRST OPEN POST-WAR AND CONTEMPORARY ART,print,50 x 63½ in. (127 x 161.2 cm.),1998,Shirley Muldowney (Pictures of Chocolate) by V...,22433.0,20707.384615,27609.846154
22282,"3,000 - 4,000 GBP",813.0,"Mar 14, 2009",Carolina Raquel Antich,Phillips London,Saturday@Phillips,Acrylic and coloured pencil on canvas.,200 x 220 cm. (78 ¾ x 86 ½ in.),2005,"Trattative by Carolina Raquel Antich, 2005",1567.0,5782.287823,7709.717097
22283,300 - 400 GBP,438.0,"Mar 14, 2009",Carolina Raquel Antich,Phillips London,Saturday@Phillips,Acrylic on canvas mounted on wooden panel.,30.5 x 37.5 cm. (12 x 14 ¾ in.),2001,"Ninna Nanna by Carolina Raquel Antich, 2001",843.0,577.397260,769.863014
22284,"2,000 - 3,000 GBP",563.0,"Mar 14, 2009",Carolina Raquel Antich,Phillips London,Saturday@Phillips,Acrylic on canvas.,150 x 230 cm. (59 x 30 1/2 in.),2005,"Volare by Carolina Raquel Antich, 2005",1085.0,3854.351687,5781.527531
22285,"2,000 - 3,000 GBP",73250.0,"Mar 14, 2009",Adriana Varejão,Phillips London,Saturday@Phillips,Oil on plaster on canvas.,Each 110.2 x 110.2 cm. (43 3/8 x 43 3/8 in.),2004,"Diptych: Azulejão - Ondas by Adriana Varejão, ...",141136.0,3853.542662,5780.313993
22286,"2,000 - 3,000 GBP",3750.0,"Mar 14, 2009",Caio Reisewitz,Phillips London,Saturday@Phillips,"C-print, Diasec mounted.",179.5 x 225.5 cm. (70 3/8 x 88 3/4 in.),2004,Camara Municipal de Sao Paulo (Palacio Anchiet...,7226.0,3853.866667,5780.800000


In [13]:
data.to_csv('../datasets/data_mutualart_completo.csv')
df.to_csv('../datasets/data_mutualart_completo_ajustado.csv')